pca_tfidf_50features 

In [3]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import warnings
warnings.filterwarnings("ignore") 

In [5]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [6]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Tfidf/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [8]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [10]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [12]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [13]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Tfidf/tfidf_withstopwordchangelevels_50_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,0,-0.792967,-0.497740,-0.123799,-0.065543,-1.013300,-0.161722,0.113425,-0.178161,-0.094483,...,-0.118447,0.011883,-0.129194,0.016643,0.090168,-0.043163,0.067717,0.054331,0.127264,0.252166
1,1,-0.952600,-0.724544,-0.173841,-0.127806,4.007133,1.862432,-1.326733,3.530465,-0.857448,...,-1.705316,-1.736966,9.821301,-6.640796,2.898908,-7.910405,-1.247435,-2.547894,-1.081622,14.786440
2,2,-0.772717,-0.484953,-0.117528,-0.065378,-0.889985,-0.163513,0.095557,-0.160656,-0.100043,...,-0.030544,0.027717,-0.066603,0.027914,0.134253,0.031764,-0.050117,0.085899,0.004546,0.162289
3,3,1.371361,2.225914,0.918214,2.448741,0.211493,0.903236,0.002741,-0.123428,1.938218,...,-3.014435,0.522942,0.863070,1.627004,-1.158215,-0.315485,0.326834,-1.383995,-0.635228,0.265971
4,4,-0.739865,-0.445717,-0.104142,-0.051366,-0.758448,-0.133752,0.072826,-0.117320,-0.073069,...,-0.012931,-0.035846,-0.004509,-0.060295,0.132503,0.241995,-0.041462,-0.008353,-0.024727,0.149233


In [14]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.792967,-0.497740,-0.123799,-0.065543,-1.013300,-0.161722,0.113425,-0.178161,-0.094483,0.090696,...,-0.118447,0.011883,-0.129194,0.016643,0.090168,-0.043163,0.067717,0.054331,0.127264,0.252166
1,-0.952600,-0.724544,-0.173841,-0.127806,4.007133,1.862432,-1.326733,3.530465,-0.857448,1.667125,...,-1.705316,-1.736966,9.821301,-6.640796,2.898908,-7.910405,-1.247435,-2.547894,-1.081622,14.786440
2,-0.772717,-0.484953,-0.117528,-0.065378,-0.889985,-0.163513,0.095557,-0.160656,-0.100043,0.066216,...,-0.030544,0.027717,-0.066603,0.027914,0.134253,0.031764,-0.050117,0.085899,0.004546,0.162289
3,1.371361,2.225914,0.918214,2.448741,0.211493,0.903236,0.002741,-0.123428,1.938218,0.867972,...,-3.014435,0.522942,0.863070,1.627004,-1.158215,-0.315485,0.326834,-1.383995,-0.635228,0.265971
4,-0.739865,-0.445717,-0.104142,-0.051366,-0.758448,-0.133752,0.072826,-0.117320,-0.073069,0.064870,...,-0.012931,-0.035846,-0.004509,-0.060295,0.132503,0.241995,-0.041462,-0.008353,-0.024727,0.149233


In [15]:
y=df['Level '] 

**Train Test Split**

In [16]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 50), (524, 50))

In [17]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [18]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [19]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3687034016752272


**Decision Tree**

In [20]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [21]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [22]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [23]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [24]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [25]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.40082,0.40082,0.542655,0.3243,0.303435,0.303435,0.427352,0.260898


**Random Forest**

In [26]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [27]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [28]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [29]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    0.9917    0.9958       361
           2     0.9982    0.9928    0.9955       553
           3     0.9808    1.0000    0.9903       306

    accuracy                         0.9943      1220
   macro avg     0.9930    0.9948    0.9939      1220
weighted avg     0.9944    0.9943    0.9943      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2535    0.1146    0.1579       157
           2     0.4442    0.7553    0.5594       237
           3     0.1600    0.0615    0.0889       130

    accuracy                         0.3912       524
   macro avg     0.2859    0.3105    0.2687       524
weighted avg     0.3165    0.3912    0.3224       524



In [30]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.400820,0.400820,0.542655,0.324300,0.303435,0.303435,0.427352,0.260898
1,RandomForest,0.994262,0.994262,0.994352,0.994275,0.391221,0.391221,0.316547,0.322361


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5478    0.3490    0.4264       361
           2     0.5234    0.8101    0.6359       553
           3     0.5821    0.2549    0.3545       306

    accuracy                         0.5344      1220
   macro avg     0.5511    0.4714    0.4723      1220
weighted avg     0.5453    0.5344    0.5033      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3235    0.2102    0.2548       157
           2     0.4598    0.7004    0.5552       237
           3     0.2131    0.1000    0.1361       130

    accuracy                         0.4046       524
   macro avg     0.3322    0.3369    0.3154       524
weighted avg     0.3578    0.4046    0.3612       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.728723,0.385991,0.440840,0.440840,0.370117,0.306182
1,RandomForest,0.994262,0.994262,0.994267,0.994264,0.402672,0.402672,0.357289,0.358558
2,Adaboost,0.534426,0.534426,0.545332,0.503344,0.404580,0.404580,0.357786,0.361226


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9401    0.8255    0.8791       361
           2     0.8124    0.9946    0.8943       553
           3     0.9779    0.7222    0.8308       306

    accuracy                         0.8762      1220
   macro avg     0.9101    0.8474    0.8681      1220
weighted avg     0.8917    0.8762    0.8739      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3010    0.1975    0.2385       157
           2     0.4484    0.6962    0.5455       237
           3     0.2075    0.0846    0.1202       130

    accuracy                         0.3950       524
   macro avg     0.3190    0.3261    0.3014       524
weighted avg     0.3445    0.3950    0.3480       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.728723,0.385991,0.440840,0.440840,0.370117,0.306182
1,RandomForest,0.994262,0.994262,0.994267,0.994264,0.402672,0.402672,0.357289,0.358558
2,Adaboost,0.534426,0.534426,0.545332,0.503344,0.404580,0.404580,0.357786,0.361226
3,GBM,0.876230,0.876230,0.891684,0.873873,0.395038,0.395038,0.344460,0.347976


**SVM Model**

In [31]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [32]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [33]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [34]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6322    0.1524    0.2455       361
           2     0.4850    0.9331    0.6382       553
           3     0.5942    0.1340    0.2187       306

    accuracy                         0.5016      1220
   macro avg     0.5704    0.4065    0.3675      1220
weighted avg     0.5559    0.5016    0.4168      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3023    0.0828    0.1300       157
           2     0.4573    0.8819    0.6023       237
           3     0.2500    0.0462    0.0779       130

    accuracy                         0.4351       524
   macro avg     0.3366    0.3369    0.2701       524
weighted avg     0.3595    0.4351    0.3307       524



In [35]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.400820,0.400820,0.542655,0.324300,0.303435,0.303435,0.427352,0.260898
1,RandomForest,0.994262,0.994262,0.994352,0.994275,0.391221,0.391221,0.316547,0.322361
2,SVM,0.501639,0.501639,0.555925,0.416791,0.435115,0.435115,0.359451,0.330699


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4958    0.4903    0.4930       361
           2     0.5690    0.7378    0.6425       553
           3     0.5616    0.2680    0.3628       306

    accuracy                         0.5467      1220
   macro avg     0.5422    0.4987    0.4995      1220
weighted avg     0.5455    0.5467    0.5281      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2800    0.2675    0.2736       157
           2     0.4598    0.6034    0.5219       237
           3     0.2857    0.1385    0.1865       130

    accuracy                         0.3874       524
   macro avg     0.3418    0.3365    0.3273       524
weighted avg     0.3627    0.3874    0.3643       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.728723,0.385991,0.440840,0.440840,0.370117,0.306182
1,RandomForest,0.994262,0.994262,0.994267,0.994264,0.402672,0.402672,0.357289,0.358558
2,Adaboost,0.534426,0.534426,0.545332,0.503344,0.404580,0.404580,0.357786,0.361226
3,GBM,0.876230,0.876230,0.891684,0.873873,0.395038,0.395038,0.344460,0.347976
4,SVM,0.508197,0.508197,0.654891,0.403714,0.450382,0.450382,0.406216,0.322108
5,KNN,0.546721,0.546721,0.545512,0.528136,0.387405,0.387405,0.362743,0.364306


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4906    0.0720    0.1256       361
           2     0.4708    0.9186    0.6225       553
           3     0.4318    0.1242    0.1929       306

    accuracy                         0.4689      1220
   macro avg     0.4644    0.3716    0.3137      1220
weighted avg     0.4669    0.4689    0.3677      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2692    0.0446    0.0765       157
           2     0.4471    0.8734    0.5914       237
           3     0.2000    0.0538    0.0848       130

    accuracy                         0.4218       524
   macro avg     0.3054    0.3239    0.2509       524
weighted avg     0.3325    0.4218    0.3115       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.728723,0.385991,0.440840,0.440840,0.370117,0.306182
1,RandomForest,0.994262,0.994262,0.994267,0.994264,0.402672,0.402672,0.357289,0.358558
2,Adaboost,0.534426,0.534426,0.545332,0.503344,0.404580,0.404580,0.357786,0.361226
3,GBM,0.876230,0.876230,0.891684,0.873873,0.395038,0.395038,0.344460,0.347976
4,SVM,0.508197,0.508197,0.654891,0.403714,0.450382,0.450382,0.406216,0.322108
5,KNN,0.546721,0.546721,0.545512,0.528136,0.387405,0.387405,0.362743,0.364306
6,GNB,0.468852,0.468852,0.466874,0.367736,0.421756,0.421756,0.332497,0.311469
